In [1]:
import pandas as pd
import numpy as np

import nltk

from nltk.data import load

In [37]:
x_train_cv = np.array([[4., 5.], [8., 9.], [7., 6.], [4., 3.]])
y_train = np.array([0, 0, 1, 1])

In [48]:
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
gnb = GaussianNB()
dt = DecisionTreeClassifier()
gnb.fit(x_train_cv, y_train)
dt.fit(x_train_cv, y_train)

DecisionTreeClassifier()

In [53]:
import pickle

pickle.dump([gnb, dt], open('example.sav', 'wb')) #saving

In [54]:
loaded_model = pickle.load(open('example.sav', 'rb'))

In [55]:
loaded_model

[GaussianNB(), DecisionTreeClassifier()]

In [58]:
classifier_algorithms = (
        ('Logistic Regression', 1),
        ('K-Nearest Neighbors', 2),
        ('Support Vector Machine', 3),
        ('Naive Bayes', 4),
        ('Decision Tree', 5),
        ('Random Forest', 6)
    )

In [17]:
gnb.predict([[5, 9]])

array([0])

In [27]:
df = pd.read_csv('Corpus/english.csv', sep='{', error_bad_lines=False)
df

,text
0,liva tuvvar siri allajatta fights against isla...
1,terrorist dogs become rebels turkey pkk twitte...
2,alkhair's gazwat is a special area after which...
3,"The defense of one Friday in Mecca, which cann..."
4,nuclear war is coming;
...,...
29331,angels plural Arab angels private Arab angels ...
29332,The first of the six conditions of faith is to...
29333,Iman Iman Iman Iman Iman Iman Iman Iman Iman I...
29334,assabuur very patient club qissa angimeler;


In [28]:
df['label'] = 0
df.iloc[:1857,:]['label'] = 1
df.iloc[1857:,:]['label'] = 0
df

c:\users\yelzha\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\users\yelzha\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,text,label
0,liva tuvvar siri allajatta fights against isla...,1
1,terrorist dogs become rebels turkey pkk twitte...,1
2,alkhair's gazwat is a special area after which...,1
3,"The defense of one Friday in Mecca, which cann...",1
4,nuclear war is coming;,1
...,...,...
29331,angels plural Arab angels private Arab angels ...,0
29332,The first of the six conditions of faith is to...,0
29333,Iman Iman Iman Iman Iman Iman Iman Iman Iman I...,0
29334,assabuur very patient club qissa angimeler;,0


In [29]:
extre = " ".join(df[df.label == 1].text.tolist())
neut = " ".join(df[df.label == 0].text.tolist())

In [77]:
import liwc
parse, category_names = liwc.load_token_parser('Corpus/LIWC2007_English100131.dic')

import re
from collections import Counter

def tokenize(text):
    # you may want to use a smarter tokenizer
    for match in re.finditer(r'\w+', text, re.UNICODE):
        yield match.group(0)

def LIWC_sum(x):
    counts = Counter(category for token in tokenize(x) for category in parse(token))
    
    result = []
    for j, tag in enumerate(category_names):
        result.append(counts[tag])
    
    return result

In [87]:
np.array(liwcdf.text.apply(lambda x: LIWC_sum(x)).values.tolist()).shape

(29336, 64)

In [82]:
liwcdf = pd.DataFrame(df)
liwcdf[category_names] = 0
liwcdf.iloc[:,2:] = liwcdf.text.apply(lambda x: LIWC_sum(x)).values.tolist()
liwcdf

,text,label,funct,pronoun,ppron,i,we,you,shehe,they,...,work,achieve,leisure,home,money,relig,death,assent,nonfl,filler
0,liva tuvvar siri allajatta fights against isla...,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,terrorist dogs become rebels turkey pkk twitte...,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,alkhair's gazwat is a special area after which...,1,6,2,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,"The defense of one Friday in Mecca, which cann...",1,12,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,nuclear war is coming;,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29331,angels plural Arab angels private Arab angels ...,0,122,18,9,0,0,0,2,7,...,1,3,0,1,0,16,0,0,0,0
29332,The first of the six conditions of faith is to...,0,90,16,9,0,6,0,3,0,...,1,10,1,0,0,1,0,0,0,0
29333,Iman Iman Iman Iman Iman Iman Iman Iman Iman I...,0,62,6,2,0,1,0,0,1,...,6,2,1,0,0,10,0,0,0,0
29334,assabuur very patient club qissa angimeler;,0,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [16]:
extre_tokens = tokenize(extre)
neut_tokens = tokenize(neut)

In [17]:
extre_counts = Counter(category for token in extre_tokens for category in parse(token))
neut_counts = Counter(category for token in neut_tokens for category in parse(token))

In [18]:
len_extre = len(extre.split(' '))
len_neut = len(neut.split(' '))

counts = []
for i in category_names:
    # print('extre', i, '=', round((extre_counts[i]/len_extre)*100, 4))
    # print('neut', i, '=', round((neut_counts[i]/len_neut)*100, 4), '\n')
    counts.append([i, round((extre_counts[i]/len_extre)*100, 6), round((neut_counts[i]/len_neut)*100, 6)])
    
catdf = pd.DataFrame(counts, columns=['category', 'extremist', 'neutral'])
catdf.index = catdf.category
catdf = catdf[['extremist', 'neutral']]
catdf

,extremist,neutral
category,,
funct,46.885221,52.117460
pronoun,7.280638,11.304945
ppron,3.863160,6.904522
i,0.204931,0.623913
we,1.025831,0.713635
...,...,...
relig,1.255424,2.489906
death,0.692304,0.342497
assent,0.049324,0.069784


### Computing linguistic information

In [21]:
def token_post_tag(x):
    from nltk.data import load
    tagdict = list(load('help/tagsets/upenn_tagset.pickle').keys())
    
    tags = nltk.pos_tag(nltk.word_tokenize(x))
    
    result = []
    for j, tag in enumerate(tagdict):
        result.append(sum([i == tag for k, i in tags]))
        
    return result

In [22]:
def checking_tagtype(x, tag):
    tags = nltk.pos_tag(nltk.word_tokenize(x))
    if len(tag) == 1:
        return sum([i == tag for k, i in tags])
    elif len(tag) > 1:
        return sum([i in tag for k, i in tags])

In [23]:
posdf.text.apply(lambda x: token_post_tag(x))

0        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 4, 0, 0, 1, ...
1        [0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 1, 0, 0, 1, ...
2        [0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 3, 1, 1, 1, ...
3        [0, 0, 2, 0, 0, 0, 1, 1, 1, 0, 0, 3, 3, 0, 1, ...
4        [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, ...
                               ...                        
29331    [0, 2, 6, 0, 4, 0, 12, 12, 6, 0, 11, 23, 33, 9...
29332    [0, 5, 5, 0, 1, 0, 2, 8, 7, 0, 1, 25, 18, 8, 1...
29333    [0, 1, 4, 0, 2, 0, 1, 6, 9, 0, 6, 71, 17, 7, 1...
29334    [0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 3, 0, 0, 1, ...
29335    [0, 11, 6, 0, 2, 0, 0, 22, 19, 0, 7, 57, 64, 1...
Name: text, Length: 29336, dtype: object

In [24]:
%%time
posdf = pd.DataFrame(df)
posdf[list(load('help/tagsets/upenn_tagset.pickle').keys())] = 0
posdf.iloc[:,2:] = posdf.text.apply(lambda x: token_post_tag(x)).values.tolist()
posdf

Wall time: 1min 10s


,text,label,LS,TO,VBN,'',WP,UH,VBG,JJ,...,MD,VB,WRB,NNP,EX,NNS,SYM,CC,CD,POS
0,liva tuvvar siri allajatta fights against isla...,1,0,0,0,0,0,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,terrorist dogs become rebels turkey pkk twitte...,1,0,0,0,0,0,0,0,3,...,0,0,0,0,0,3,0,0,0,0
2,alkhair's gazwat is a special area after which...,1,0,1,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,0,1
3,"The defense of one Friday in Mecca, which cann...",1,0,0,2,0,0,0,1,1,...,1,1,0,3,0,0,0,0,1,0
4,nuclear war is coming;,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29331,angels plural Arab angels private Arab angels ...,0,0,2,6,0,4,0,12,12,...,1,8,1,28,0,28,0,10,1,0
29332,The first of the six conditions of faith is to...,0,0,5,5,0,1,0,2,8,...,4,8,0,5,0,4,0,6,2,0
29333,Iman Iman Iman Iman Iman Iman Iman Iman Iman I...,0,0,1,4,0,2,0,1,6,...,1,3,0,63,1,8,0,3,4,0
29334,assabuur very patient club qissa angimeler;,0,0,0,0,0,0,0,0,2,...,0,0,0,0,0,0,0,0,0,0


In [145]:
test = pd.DataFrame([0])
test[list(load('help/tagsets/upenn_tagset.pickle').keys())] = 0
test.loc[0][1:] = token_post_tag('liva tuvvar siri allajatta fights against')
test

c:\users\yelzha\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
c:\users\yelzha\appdata\local\programs\python\python37\lib\site-packages\pandas\core\series.py:1043: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.iloc[indexer] = extract_array(value, extract_numpy=True)


,0,LS,TO,VBN,'',WP,UH,VBG,JJ,VBZ,...,MD,VB,WRB,NNP,EX,NNS,SYM,CC,CD,POS
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [117]:
test[list(load('help/tagsets/upenn_tagset.pickle').keys())] = 0
test

,LS,TO,VBN,'',WP,UH,VBG,JJ,VBZ,--,...,NNP,EX,NNS,SYM,CC,CD,POS,awd,awdawdawdwa,12
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
nltk.pos_tag(nltk.word_tokenize('liva tuvvar siri allajatta fights against'))

[('liva', 'NN'),
 ('tuvvar', 'NN'),
 ('siri', 'NN'),
 ('allajatta', 'JJ'),
 ('fights', 'NNS'),
 ('against', 'IN')]

In [16]:
feature = pd.DataFrame(np.vstack((df.label, df.text)).T, columns=['label', 'text'])
feature['text_word_count'] = feature.text.apply(len)
feature['personal_pronoun_count'] = feature.text.apply(lambda x: checking_tagtype(x, 'PRP'))
feature['personal_pronoun_probability'] = feature.personal_pronoun_count/feature.text_word_count
feature

,label,text,text_word_count,personal_pronoun_count,personal_pronoun_probability
0,1,liva tuvvar siri allajatta fights against isla...,53,0,0.000000
1,1,terrorist dogs become rebels turkey pkk twitte...,61,0,0.000000
2,1,alkhair's gazwat is a special area after which...,69,1,0.014493
3,1,"The defense of one Friday in Mecca, which cann...",134,0,0.000000
4,1,nuclear war is coming;,22,0,0.000000
...,...,...,...,...,...
29331,0,angels plural Arab angels private Arab angels ...,1428,9,0.006303
29332,0,The first of the six conditions of faith is to...,747,8,0.010710
29333,0,Iman Iman Iman Iman Iman Iman Iman Iman Iman I...,1505,7,0.004651
29334,0,assabuur very patient club qissa angimeler;,43,0,0.000000


In [6]:
print('Neutral personal noun summary probability: %.6f' % (feature[feature.label == 0].personal_pronoun_count.sum()/feature[feature.label == 0].text_word_count.sum()))
print('Extra personal noun summary probability: %.6f' % (feature[feature.label == 1].personal_pronoun_count.sum()/feature[feature.label == 1].text_word_count.sum()))

Neutral personal noun summary probability: 0.013780
Extra personal noun summary probability: 0.006832


In [ ]:
sum([i in tag for i in feature.text.tolist()[29335]])

In [32]:
feature['future_verb_count'] = feature.text.apply(lambda x: checking_tagtype(x, ["VP"]))
feature['future_verb_probability'] = feature['future_verb_count']/feature['text_word_count']
feature

,label,text,text_word_count,personal_pronoun_count,personal_pronoun_probability,future_verb_count,future_verb_probability
0,1,liva tuvvar siri allajatta fights against isla...,53,0,0.000000,0,0.0
1,1,terrorist dogs become rebels turkey pkk twitte...,61,0,0.000000,0,0.0
2,1,alkhair's gazwat is a special area after which...,69,1,0.014493,0,0.0
3,1,"The defense of one Friday in Mecca, which cann...",134,0,0.000000,0,0.0
4,1,nuclear war is coming;,22,0,0.000000,0,0.0
...,...,...,...,...,...,...,...
29331,0,angels plural Arab angels private Arab angels ...,1428,9,0.006303,0,0.0
29332,0,The first of the six conditions of faith is to...,747,8,0.010710,0,0.0
29333,0,Iman Iman Iman Iman Iman Iman Iman Iman Iman I...,1505,7,0.004651,0,0.0
29334,0,assabuur very patient club qissa angimeler;,43,0,0.000000,0,0.0


In [33]:
print('Neutral future verb summary probability: %.6f' % (feature[feature.label == 0].future_verb_count.sum()/feature[feature.label == 0].text_word_count.sum()))
print('Extra future verb summary probability: %.6f' % (feature[feature.label == 1].future_verb_count.sum()/feature[feature.label == 1].text_word_count.sum()))

Neutral future verb summary probability: 0.000000
Extra future verb summary probability: 0.000000


In [205]:
from googletrans import Translator
translator = Translator()

In [206]:
aggressive_words = pd.read_csv('Corpus/aggressive_word.csv')
aggressive_words.head()

,word
0,hostile
1,predatory
2,belligerent
3,violence
4,competitive


In [207]:
aggressive_words.shape

(579, 1)

In [208]:
agg_words = translator.translate(' '.join(aggressive_words.word.tolist()), dest='kk').text.split(' ')
agg_words[:10]

['дұшпандық',
 'жыртқыш',
 'ұрыс-керіс',
 'зорлық-зомбылық',
 'жекпе-жек',
 'қарулы',
 'жауынгер',
 'шайқас',
 'ұрыс-керіс',
 'мылжың']

In [209]:
feature['aggressive_words_count'] = feature.text.apply(lambda x: sum([True if i in x.split(' ') else False for i in agg_words]))
feature['aggressive_words_probability'] = feature.aggressive_words_count/feature.text_word_count
feature

,label,text,text_word_count,personal_noun_count,personal_noun_probability,future_verb_count,future_verb_probability,aggressive_words_count,aggressive_words_probability
0,1,лива туввар сирия аллажатта islamstateқа қарсы...,61,2,0.032787,5,0.081967,13,0.213115
1,1,террористік иттер бүлікшілерге айналады turke...,73,1,0.013699,2,0.027397,0,0.000000
2,1,әлхайрдың газваты арнайы аймақ олардан кейін о...,74,3,0.040541,3,0.040541,0,0.000000
3,1,сириядағы ресей n басып кіру туралы алаңдамауд...,125,5,0.040000,5,0.040000,6,0.048000
4,1,ядролық соғыс соғысы келе жатыр,32,2,0.062500,3,0.093750,32,1.000000
...,...,...,...,...,...,...,...,...,...
29331,0,періште көпше араб малайкат жекеше араб малайк...,1683,50,0.029709,67,0.039810,17,0.010101
29332,0,иманның алты шартының ең әуелгісі алла тағалағ...,884,28,0.031674,29,0.032805,6,0.006787
29333,0,"иман иман араб сенім сену, илану, қабылдау алл...",1729,65,0.037594,51,0.029497,11,0.006362
29334,0,ассабуур аса сабырлы club qissa angimeler,43,3,0.069767,2,0.046512,0,0.000000


In [210]:
print('Neutral aggressive word summary probability: %.6f' % (feature[feature.label == 0].aggressive_words_count.sum()/feature[feature.label == 0].text_word_count.sum()))
print('Extra aggressive word summary probability: %.6f' % (feature[feature.label == 1].aggressive_words_count.sum()/feature[feature.label == 1].text_word_count.sum()))

Neutral aggressive word summary probability: 0.008312
Extra aggressive word summary probability: 0.014471


In [211]:
feature['nonaggressive_words_count'] = feature.text_word_count - feature.aggressive_words_count
feature['nonaggressive_words_probability'] = feature.nonaggressive_words_count/feature.text_word_count
feature

,label,text,text_word_count,personal_noun_count,personal_noun_probability,future_verb_count,future_verb_probability,aggressive_words_count,aggressive_words_probability,nonaggressive_words_count,nonaggressive_words_probability
0,1,лива туввар сирия аллажатта islamstateқа қарсы...,61,2,0.032787,5,0.081967,13,0.213115,48,0.786885
1,1,террористік иттер бүлікшілерге айналады turke...,73,1,0.013699,2,0.027397,0,0.000000,73,1.000000
2,1,әлхайрдың газваты арнайы аймақ олардан кейін о...,74,3,0.040541,3,0.040541,0,0.000000,74,1.000000
3,1,сириядағы ресей n басып кіру туралы алаңдамауд...,125,5,0.040000,5,0.040000,6,0.048000,119,0.952000
4,1,ядролық соғыс соғысы келе жатыр,32,2,0.062500,3,0.093750,32,1.000000,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
29331,0,періште көпше араб малайкат жекеше араб малайк...,1683,50,0.029709,67,0.039810,17,0.010101,1666,0.989899
29332,0,иманның алты шартының ең әуелгісі алла тағалағ...,884,28,0.031674,29,0.032805,6,0.006787,878,0.993213
29333,0,"иман иман араб сенім сену, илану, қабылдау алл...",1729,65,0.037594,51,0.029497,11,0.006362,1718,0.993638
29334,0,ассабуур аса сабырлы club qissa angimeler,43,3,0.069767,2,0.046512,0,0.000000,43,1.000000


In [212]:
print('Neutral non-aggressive word summary probability: %.6f' % (feature[feature.label == 0].nonaggressive_words_probability.sum()/feature[feature.label == 0].shape[0]))
print('Extra non-aggressive word summary probability: %.6f' % (feature[feature.label == 1].nonaggressive_words_probability.sum()/feature[feature.label == 1].shape[0]))

Neutral non-aggressive word summary probability: 0.989227
Extra non-aggressive word summary probability: 0.977458


In [213]:
print('Neutral non-aggressive word summary probability: %.6f' % (feature[feature.label == 0].nonaggressive_words_probability.sum()/feature[feature.label == 0].nonaggressive_words_count.sum()))
print('Extra non-aggressive word summary probability: %.6f' % (feature[feature.label == 1].nonaggressive_words_probability.sum()/feature[feature.label == 1].nonaggressive_words_count.sum()))

Neutral non-aggressive word summary probability: 0.003719
Extra non-aggressive word summary probability: 0.001448
